In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
print('Pytorch version :', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device : [{device}]')

Pytorch version : 1.7.1
device : [cuda:0]


In [27]:
class RNN(nn.Module):
    def __init__(self,x_dim=28,h_dim=256,y_dim=10,n_layer=1 ):
        super(RNN,self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.y_dim = y_dim
        self.n_layer = n_layer
        
        self.rnn = nn.RNN(
        input_size=self.x_dim,hidden_size=self.h_dim,num_layers=self.n_layer,batch_first=True)
        self.init_param()
    
    def forward(self,input):
        x = input
        h_0 = torch.zeros(self.n_layer,x.size(0),self.h_dim).to(device)
        c_0 = torch.zeros(self.n_layer,x.size(0),self.h_dim).to(device)
        rnn_out, (h_n,c_n) = self.rnn(x,(h_0,c_0))
        out = self.lin(rnn_out[:,-1,:]).view(-1,self.y_dim)
        return out
    
    def init_param(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.kaiming_normal_(param.data)
            elif 'bias' in name:
                nn.init.zeros_(param.data)

In [28]:
a = RNN(x_dim=28,h_dim=256,y_dim=10,n_layer=1)

In [29]:
list(a.modules())

[RNN(
   (rnn): RNN(28, 256, batch_first=True)
 ),
 RNN(28, 256, batch_first=True)]

In [30]:
for name,param in a.named_parameters():
    print(name)
    print(param.size())
    
    

rnn.weight_ih_l0
torch.Size([256, 28])
rnn.weight_hh_l0
torch.Size([256, 256])
rnn.bias_ih_l0
torch.Size([256])
rnn.bias_hh_l0
torch.Size([256])
